In [4]:
import os
import time
import random
import collections

import numpy as np
import tensorflow as tf

In [5]:
tf.__version__

'2.4.0'

In [6]:
# ট্রেনিং প্যারামিটার 
LEARNING_RATE = 0.1
BATCH_SIZE = 1024
NUM_STEPS = 400000
DISPLAY_STEPS = 10000
EVAL_STEPS = 200000

# এই শব্দগুলো দিয়ে আমরা মডেল ট্রেনিং করার সময় টেস্ট করে দেখব যে মডেল কেমন শিখতেছে
EVAL_WORDS = ['মুনা', 'বকুল', 'বাকের']

# এম্বেডিং ভেক্টরে ডাইমেসন। একটা শব্দের বিপরীতে ২০০ টা স্কেলার ভেলু জমা থাকবে 
# এবং এটাই হবে ওই শব্দের ভেক্টর রিপ্রেজেন্টেসন
EMBEDDING_SIZE = 200

# আমাদের করপাস থেকে এর থেকে বেশি শব্দ থাকলে সেগুলো কে বাদ দেব । 
MAX_VOCABULARY_SIZE = 50000

# যে শব্দগুলো আমাদের ডাটা তে খুব কম সংখ্যক বার আছে সেগুলো আমারা বাদ দিব ।  
MIN_OCCURRENCE = 10

# যখন আমরা ডাটা বানাব তখন আমাদের লেফট এবং রাইট সাইট এর কতগুলো শব্দও ব্যাবহার করব
SKIP_WINDOW = 3

# আমরা একই শব্দ গিয়ে কত গুলো ডাটা বানাতে চাই তার সংখ্যা
NUM_SKIPS = 2

# লস হিসাব করার সময় কতগুলো নেগেটিভ ডাটা ব্যাবহার করতে চাই 
NUM_SAMPLED = 64

In [8]:
corpus = '/content/kothou_keu_nei_v1.2.txt'

unique_chars = set()
with open(corpus, 'r') as fp:
    lines = fp.readlines()
    # find unique chars so that we can remove unwanted chars
    for line in lines:
        line = line.strip()
        if line:
            for c in line:
                unique_chars.add(c)
    unique_chars = sorted(unique_chars)
    print("Unique chars:", unique_chars)

Unique chars: [' ', '!', '(', ')', ',', '-', '.', '1', ':', ';', '<', '?', 'R', 'r', '{', '·', 'я', 'अ', 'छ', 'ब', 'भ', 'य', 'र', 'ल', 'श', 'ा', 'ि', '्', '।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'ঢ়', 'য়', '১', '২', '৩', '৪', '৫', '৬', '৭', 'ৰ', 'ৱ', '৷', '–', '—', '‘', '’', '“', '”', '…']


In [9]:
def clean(filter_list, text):
    for ch in filter_list:
        text = text.replace(ch, '')
    text = text.strip()
    return text

# for dataset version 1.2
chars_to_remove = [
    '!', '(', ')', ',', '-', '.', '1', ':', ';', '<', '?', 'R', 'r', '{', '·',
    'я', 'अ', 'छ', 'ब', 'भ', 'य', 'र', 'ल', 'श', 'ৰ', 'ৱ', '৷', '–', '—', 
    '‘', '’', '“', '”', '…', '।'
]

text_words = []
for line in lines:
    text = clean(chars_to_remove, line)
    if text and text != '':
        words = text.split()
        text_words.extend(words)

print("Total words:", len(text_words))

Total words: 80985


In [10]:
# Unknown ওয়ার্ড এর জন্য UNK টকেন ব্যাবহার করব ।
dictionary = [('UNK', -1)]
# MAX_VOCABULARY_SIZE ধরে কমন (মানে যে শব্দও গুলোর ফ্রিকুয়েন্সি বেশি) সেগুলো নিয়ে নিলাম 
dictionary.extend(
    collections.Counter(text_words).most_common(MAX_VOCABULARY_SIZE - 1)
)

print("Total unique words:", len(dictionary))
print("Dictionary of unique words", dictionary)

Total unique words: 9275
Dictionary of unique words [('UNK', -1), ('না', 2683), ('করে', 919), ('তার', 750), ('বলল', 742), ('কি', 741), ('মুনা', 709), ('সে', 620), ('একটা', 593), ('আছে', 556), ('কিছু', 551), ('তো', 502), ('কথা', 495), ('হয়', 479), ('হয়ে', 469), ('হবে', 445), ('বকুল', 440), ('বাকের', 434), ('আমি', 432), ('এই', 431), ('কেন', 430), ('সঙ্গে', 398), ('নিয়ে', 383), ('ভাল', 373), ('মনে', 370), ('আমার', 370), ('নেই', 365), ('সাহেব', 360), ('খুব', 305), ('হচ্ছে', 293), ('গেল', 290), ('বললেন', 289), ('দিয়ে', 282), ('তুমি', 282), ('কী', 274), ('হয়েছে', 270), ('থেকে', 269), ('করতে', 267), ('আর', 265), ('এক', 258), ('কিন্তু', 258), ('বসে', 248), ('কোনো', 247), ('বড়', 242), ('আপা', 240), ('হল', 239), ('যে', 235), ('মামুন', 234), ('এখন', 218), ('জন্যে', 218), ('নাকি', 217), ('এ', 216), ('রকম', 214), ('সময়', 213), ('করছে', 212), ('তিনি', 209), ('যায়', 207), ('বাবু', 206), ('চলে', 196), ('সব', 195), ('তাকে', 194), ('ঠিক', 193), ('ভাই', 190), ('চা', 190), ('শওকত', 189), ('গলায়', 183), ('

In [11]:
# Remove samples with less than 'min_occurrence' occurrences.
# যে শব্দ গুলো 'MIN_OCCURRENCE' থেকে ছোট সেগুলোকে আমরা বাদ দিয়ে দেই
for i in range(len(dictionary)-1, -1, -1):
    # i'th শব্দ কতবার এসেছে সেটা নেই
    occurrence = dictionary[i][1]
    # যদি MIN_OCCURRENCE থেকে ছোট হয় তাহলে আমরা এই শব্দকে dictionary থেক বাদ দেই
    if occurrence < MIN_OCCURRENCE:
        dictionary.pop(i)
    else:
        # যেহেতু আমাদের dictionary টা occurrence এর উপর ভিত্তি করে সর্ট (বেশি থেকে কম) করা 
        # এবং আমরা এটাকে শেষ থেকে শুরুর দিকে ইতারেট করতেছি এই জন্য আমরা যদি কোন শব্দের জন্য
        # occurrence < MIN_OCCURRENCE না পাই তাহলে আমরা লুপ থেকে বের হয়ে যাব। কারণ সবগুলো
        # অলরেডি বাদ পরে গেছে । 
        break

VOCABULARY_SIZE = len(dictionary)
print("Vocabulary size after filter min occurrence word:", VOCABULARY_SIZE)

Vocabulary size after filter min occurrence word: 1185


In [12]:
word2id = {}
id2word = {}
for i, (word, _)in enumerate(dictionary):
    # প্রত্যেক টা শব্দকে আমরা একটা আইডি এসাইন করে দেই
    word2id[word] = i
    # আইডি থেকে আবার ওয়ার্ড এ ফিরে আসার জন্য ট্রাক রাখি
    id2word[i] = word
print('word2id:', word2id)
print('id2word:', id2word)

word2id: {'UNK': 0, 'না': 1, 'করে': 2, 'তার': 3, 'বলল': 4, 'কি': 5, 'মুনা': 6, 'সে': 7, 'একটা': 8, 'আছে': 9, 'কিছু': 10, 'তো': 11, 'কথা': 12, 'হয়': 13, 'হয়ে': 14, 'হবে': 15, 'বকুল': 16, 'বাকের': 17, 'আমি': 18, 'এই': 19, 'কেন': 20, 'সঙ্গে': 21, 'নিয়ে': 22, 'ভাল': 23, 'মনে': 24, 'আমার': 25, 'নেই': 26, 'সাহেব': 27, 'খুব': 28, 'হচ্ছে': 29, 'গেল': 30, 'বললেন': 31, 'দিয়ে': 32, 'তুমি': 33, 'কী': 34, 'হয়েছে': 35, 'থেকে': 36, 'করতে': 37, 'আর': 38, 'এক': 39, 'কিন্তু': 40, 'বসে': 41, 'কোনো': 42, 'বড়': 43, 'আপা': 44, 'হল': 45, 'যে': 46, 'মামুন': 47, 'এখন': 48, 'জন্যে': 49, 'নাকি': 50, 'এ': 51, 'রকম': 52, 'সময়': 53, 'করছে': 54, 'তিনি': 55, 'যায়': 56, 'বাবু': 57, 'চলে': 58, 'সব': 59, 'তাকে': 60, 'ঠিক': 61, 'ভাই': 62, 'চা': 63, 'শওকত': 64, 'গলায়': 65, 'আপনি': 66, 'গেছে': 67, 'খারাপ': 68, 'তোমার': 69, 'কেমন': 70, 'কোন': 71, 'মত': 72, 'এত': 73, 'আজ': 74, 'যেন': 75, 'এসে': 76, 'ছিল': 77, 'কাছে': 78, 'করল': 79, 'যাবে': 80, 'আপনার': 81, 'বলে': 82, 'ঘরে': 83, 'আবার': 84, 'দেখে': 85, 'বকুলের': 86, 'কেউ': 87

In [13]:
data = []
unk_word_count = 0
for word in text_words:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in dictionary.
    # text_words এর প্রতিটি শব্দকে একটা আইডি দিয়ে দেই ।
    # যদি শব্দটা টা word2id মধ্যে পাওয়া না যায় তাহলে এটা একটা UNK ক্যাটাগরির শব্দ 
    # এই জন্য এটাকে আমরা ০ আইডি দিয়ে দিব 
    index = word2id.get(word, 0)
    
    # UNK টকেনের ফ্রিকুয়েন্সি কেমন হয় সেটা ট্রাক রাখা
    if index == 0:
        unk_word_count += 1
    # প্রতিটা শব্দের বিপরীতে আমরা সেই শব্দের আইডি কে রাখলাম এবং এই ডাটা দিয়েই আমরা আমাদের মডেল 
    # ট্রেনিং করব । 
    data.append(index)
# UNK এর ফ্রিকুয়েন্সি ডিকশনারিতে এড করে দিলাম
dictionary[0] = ('UNK', unk_word_count)

# শব্দকে আইডি দেয়ার পরের অবস্থা 
print("data        :", data)
print("Totla data  :", len(data))
print('-' * 50)
# আইডি থেকে আবার শব্দে ফিরে আসার পরের অবস্থা
print("data in text:", [id2word[id] for id in data])
print('-' * 50)


print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", VOCABULARY_SIZE)
print("Most common words:", dictionary[:20])

data        : [962, 78, 76, 6, 764, 167, 260, 79, 0, 73, 350, 114, 103, 30, 1, 0, 103, 56, 1, 38, 48, 11, 397, 245, 36, 0, 148, 764, 0, 731, 1024, 0, 0, 36, 51, 192, 207, 28, 211, 180, 963, 964, 0, 808, 48, 351, 0, 0, 107, 10, 187, 150, 265, 106, 809, 92, 0, 965, 187, 35, 3, 139, 64, 27, 149, 601, 0, 107, 504, 379, 75, 246, 0, 0, 538, 7, 52, 303, 6, 853, 398, 28, 732, 325, 561, 0, 0, 0, 14, 9, 672, 334, 1025, 0, 0, 0, 123, 7, 896, 261, 0, 0, 254, 0, 0, 0, 310, 1105, 15, 285, 0, 310, 966, 304, 733, 35, 148, 0, 626, 318, 38, 0, 80, 1, 6, 254, 659, 659, 732, 0, 90, 539, 367, 196, 93, 311, 0, 0, 430, 562, 0, 0, 0, 0, 108, 0, 34, 15, 16, 0, 0, 0, 289, 0, 539, 765, 132, 68, 93, 6, 24, 24, 0, 37, 260, 79, 0, 0, 0, 34, 15, 0, 0, 1, 157, 10, 0, 286, 7, 854, 380, 0, 1025, 0, 247, 0, 87, 312, 208, 1, 6, 275, 89, 563, 16, 19, 16, 16, 368, 368, 474, 475, 99, 64, 27, 0, 855, 8, 0, 37, 431, 133, 0, 0, 19, 0, 34, 108, 16, 368, 368, 4, 235, 6, 44, 176, 64, 27, 380, 65, 31, 105, 700, 96, 0, 247, 0, 284,

In [19]:
data_index = 0
# Generate training batch for the skip-gram model.
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # get window size (words left and right + current one).
    span = 2 * skip_window + 1
    # difine a deque buffer to store span words
    # buffer words will change over iteration
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    # select span of words
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    # prepare batch data
    for i in range(batch_size // num_skips):
        # for span = 7 and skip_window = 3
        # span range = [0, 1, 2, 3, 4, 5, 6]
        # 3 will be discard 
        # context_word = [0, 1, 2, 4, 5, 6]
        context_words = [w for w in range(span) if w != skip_window]
        # randomly take `num_skips` sample from context_word
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            # j = 0 মুনা, j = 1 মুন
            batch[i * num_skips + j] = buffer[skip_window]
            # j = 0 গেটের, j = 1 এসে
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            # if we reach end to take '0 to span' words
            buffer.extend(data[0:span])
            # update data index
            data_index = span
        else:
            # update buffer by new word
            buffer.append(data[data_index])
            data_index += 1
    # শেষের শব্দগুলো যাতে বাদ না পরে এই জন্য ইনডেক্স টা আপডেট করে দিলাম
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [20]:
batch_x, batch_y = next_batch(BATCH_SIZE, NUM_SKIPS, SKIP_WINDOW)
print("shape target:", batch_x.shape)
print('shape context:', batch_y.shape)
print('-' * 50)

print('target word :', end='')
for i in range(len(batch_x)):
    print(id2word[batch_x[i]], end=',')
print('')
print('-' * 50)

print("context word:", end='')
for i in range(len(batch_x)):
    print(id2word[batch_y[i][0]], end=',')


shape target: (1024,)
shape context: (1024, 1)
--------------------------------------------------
target word:মুনা,মুনা,ঘড়ি,ঘড়ি,দেখতে,দেখতে,চেষ্টা,চেষ্টা,করল,করল,UNK,UNK,এত,এত,ছোট,ছোট,কিছুই,কিছুই,দেখা,দেখা,গেল,গেল,না,না,UNK,UNK,দেখা,দেখা,যায়,যায়,না,না,আর,আর,এখন,এখন,তো,তো,অন্ধকার,অন্ধকার,রিকশা,রিকশা,থেকে,থেকে,UNK,UNK,একবার,একবার,ঘড়ি,ঘড়ি,UNK,UNK,সাড়ে,সাড়ে,সাত,সাত,UNK,UNK,UNK,UNK,থেকে,থেকে,এ,এ,পর্যন্ত,পর্যন্ত,আসতে,আসতে,খুব,খুব,বেশি,বেশি,হলে,হলে,চার,চার,মিনিট,মিনিট,UNK,UNK,কাজেই,কাজেই,এখন,এখন,বাজে,বাজে,UNK,UNK,UNK,UNK,এমন,এমন,কিছু,কিছু,রাত,রাত,হয়নি,হয়নি,তবু,তবু,মুনার,মুনার,অস্বস্তি,অস্বস্তি,লাগছে,লাগছে,UNK,UNK,ফিরতে,ফিরতে,রাত,রাত,হয়েছে,হয়েছে,তার,তার,মামা,মামা,শওকত,শওকত,সাহেব,সাহেব,একটি,একটি,কথাও,কথাও,UNK,UNK,এমন,এমন,ভাব,ভাব,করেছেন,করেছেন,যেন,যেন,মুনাকে,মুনাকে,UNK,UNK,UNK,UNK,আজও,আজও,সে,সে,রকম,রকম,করবেন,করবেন,মুনা,মুনা,গেট,গেট,খুলে,খুলে,খুব,খুব,সাবধানে,সাবধানে,ভেতরে,ভেতরে,ঢুকল,ঢুকল,UNK,UNK,UNK,UNK,UNK,UNK,হয়ে,হয়ে,আছে,আছে,সকালে,সকালে,বাবুকে,বাবুকে,দুবার,দুবার,UNK,UNK,UNK,UNK,UNK,UNK,দিতে,দিত

In [22]:

# এই ভেরিয়েবলে আমরা আমাদের এম্বিডিং কে স্টোর করে রাখবে
embedding = tf.Variable(tf.random.normal([VOCABULARY_SIZE, EMBEDDING_SIZE]))

# এই ভেরিয়েবল গুলো NCE loss হিসাব করার সময় ব্যাবহার হবে 
nce_weights = tf.Variable(tf.random.normal([VOCABULARY_SIZE, EMBEDDING_SIZE]))
nce_biases = tf.Variable(tf.zeros([VOCABULARY_SIZE]))

In [25]:
def get_embedding(word):
    # প্রতিটা ওয়ার্ড এর বিপরীতে 'embedding' ভেরিয়েবলে যে embedding স্টোর করা আছে সেটা বের করা 
    x_embed = tf.nn.embedding_lookup(embedding, word)
    return x_embed

def nce_loss(x_embed, y):
    # একটা ব্যাচ ডাটার বিপরীতে NCE loss হিসাব করা
    y = tf.cast(y, tf.int64)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=y,
            inputs=x_embed,
            num_sampled=NUM_SAMPLED, # number of negative sample used
            num_classes=VOCABULARY_SIZE
        )
    )
    return loss


def cosing_similarity(x_embed):
    # ইনপুট এম্বেডিং ভেক্টরের সাথে প্রতিটা এম্বেডিং ভেক্টরের cosine similarity বের করা
    x_embed = tf.cast(x_embed, tf.float32)
    x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
    embedding_norm = embedding / tf.sqrt(
        tf.reduce_sum(tf.square(embedding), 1, keepdims=True), tf.float32
    )
    cosine_sim_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)
    return cosine_sim_op

# আমরা SGD optimizer ব্যাবহার করব 
optimizer = tf.optimizers.SGD(LEARNING_RATE)

In [32]:
x_test = np.array([word2id[w] for w in EVAL_WORDS])

start_time = time.time()
for step in range(1, NUM_STEPS+1):
    # ট্রেনিং এর জন্য ডাটা নেই 
    target_x, context_y = next_batch(BATCH_SIZE, NUM_SKIPS, SKIP_WINDOW)
    
    with tf.GradientTape() as tape:
        # টার্গেট শব্দগুলোর বর্তমান এম্বেডিং বের করি
        emb = get_embedding(target_x)
        # এম্বিডিং এবং কন্টেক্সট থেকে লস হিসাব করি । 
        loss = nce_loss(emb, context_y)
    # loss এর সাপেক্ষে embedding, nce_weights, nce_biases ভেরিয়েবল গুলোর গ্রাডিয়েন্ট হিসাব করা 
    gradients = tape.gradient(loss, [embedding, nce_weights, nce_biases])
    # এই গ্রাডিয়েন্ট ধরে আমরা embedding, nce_weights, nce_biases ভেরিয়েবল গুলোর ভেলু আপডেট
    # করি 
    optimizer.apply_gradients(zip(gradients, [embedding, nce_weights, nce_biases]))
    
    # নিদিষ্ট স্টেপ পর পর আমরা লস দেখব 
    if step % DISPLAY_STEPS == 0 or step == 1:
        temp_emb = get_embedding(batch_x)
        loss = nce_loss(temp_emb, batch_y)
        print("Step: {} loss: {:.4f} time: {}".format(
            step, loss, time.time()-start_time)
        )
        start_time = time.time()
        
    # আমাদের সেট করা কিছু টেস্ট শব্দ দিয়ে আমরা টেস্ট করে দেখব আমাদের মডেল কেমন শিখতেছে
    if step % EVAL_STEPS == 0 or step == 1:
        print("Testing...")
        similarity = cosing_similarity(get_embedding(x_test)).numpy()
        for i in range(len(EVAL_WORDS)):
            top_k = 8  # আমরা কতগুলো নেয়ারেস্ট শব্দ দেখতে চাই সেটা সেট করে দিলাম 
            nearest = (-similarity[i, :]).argsort()[1:top_k+1]
            log_str = "'{}' এর কাছের শব্দ গুলো: ".format(EVAL_WORDS[i])
            for k in range(top_k):
                log_str = "{} {},".format(log_str, id2word[nearest[k]])
            print(log_str)


Step: 1 loss: 115.20759582519531 time: 0.0004235943158467611
Testing...
'মুনা' এর কাছের শব্দ গুলো:  না, UNK, করে, হয়, তার, বলল, আছে, আমার,
'বকুল' এর কাছের শব্দ গুলো:  করে, তো, তুমি, হয়, কিছু, না, শুয়ে, UNK,
'বাকের' এর কাছের শব্দ গুলো:  সে, না, কিছু, UNK, তো, কথা, এই, মুনা,
Step: 10000 loss: 9.455401420593262 time: 2.2726904074350993
Step: 20000 loss: 7.710119724273682 time: 2.2709779103597003
Step: 30000 loss: 7.070896148681641 time: 2.2607158144315083
Step: 40000 loss: 6.505194664001465 time: 2.2212836742401123
Step: 50000 loss: 6.173004150390625 time: 2.214940134684245
Step: 60000 loss: 5.946339130401611 time: 2.2243704438209533
Step: 70000 loss: 5.727808952331543 time: 2.221015910307566
Step: 80000 loss: 5.547652244567871 time: 2.1990910172462463
Step: 90000 loss: 5.408536434173584 time: 2.1639158805211385
Step: 100000 loss: 5.279882431030273 time: 2.123226515452067
Step: 110000 loss: 5.184957027435303 time: 2.106153655052185
Step: 120000 loss: 5.0996479988098145 time: 2.10821903546

In [34]:
# Save only vector to a tsv file
# tensorflow projector only accepts this format of data
with open('kothou_keu_nei_vector.tsv', 'w') as fp:
    for i in range(VOCABULARY_SIZE):
        # i তম শব্দের জন্য এম্বেডিং
        embed = embedding[i, :]
        # convert embedding to numpy and save it to a file
        fp.write('{}\n'.format('\t'.join(map(str, embed.numpy()))))

# Save all vocabulary to a file
with open('kothou_keu_nei_metadata.tsv', 'w') as fp:
    for i in range(VOCABULARY_SIZE):
        # take word from word id
        word = id2word[i]
        # write it to a file
        fp.write('{}\n'.format(word))


## Visualize Learned Embeddings: 

Load vector and metadata tsv to projector 
- [https://projector.tensorflow.org/](https://projector.tensorflow.org/)
- You can also user tensorboard `projector` tab

## Resoruces
- [Efficient Estimation of Word Representations inVector Space](https://arxiv.org/pdf/1301.3781.pdf)
- [Skipgram with examples](https://www.tensorflow.org/tutorials/text/word2vec#skip-gram_and_negative_sampling)
- [Word2Vec Tensorflow 2x with low level api](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/tensorflow_v2/notebooks/2_BasicModels/word2vec.ipynb)
- [Word2Vec using Embedding Layer Tensorflow 2x](https://petamind.com/word2vec-with-tensorflow-2-0-a-simple-cbow-implementation/)
- [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec)
- [Word Embeddings](https://www.tensorflow.org/tutorials/text/word_embeddings)
- [skipgram function defination](https://keras.rstudio.com/reference/skipgrams.html)